In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 03
### Team Member Names: Sherry, Kelly, Ifan
### Team Strategy Chosen: SAFE

In [2]:
tickers_df = pd.read_csv('Tickers_Example.csv')
# When the CSV file is read, it takes the first ticker and makes it the name of the column, so we need to extract it
tickers_list = list(tickers_df.columns)
# Now add the rest of the tickers in after it:
tickers_list += tickers_df.iloc[:, 0].tolist()
tickers_num = len(tickers_list)

# Dates to pull closing data from, change as needed
start_date = '2020-01-01'
end_date = '2022-01-02'

# Dictionary of all the stocks
stocks_dict = {}
stocks_full_info = {}

# Function produces the daily closing prices of a stock
def get_closing_prices(ticker, start, end):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start, end=end) #Daily
    return hist.Close

for i in range (tickers_num):
    stocks_dict[tickers_list[i]] = get_closing_prices(tickers_list[i], start_date, end_date)
    stocks_full_info[tickers_list[i]] = yf.Ticker(tickers_list[i])

# Create DataFrame
all_prices = pd.DataFrame(stocks_dict)

#Some of the stocks are delisted, so if we uncomment this now, it'll delete the entire DataFrame
#prices = prices.dropna() 

all_prices

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- RTN: No data found, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted


,AAPL,ABBV,ABT,ACN,AGN,AIG,AMZN,AXP,BA,BAC,...,SPG,T,TD.TO,TGT,TWX,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,73.561539,77.932625,82.906250,201.554703,NaN,47.586327,94.900497,121.071251,331.348572,33.473267,...,123.889420,22.740847,64.691452,119.371849,NaN,119.591988,281.049622,172.249115,107.104683,53.313934
2020-01-03,72.846367,77.192886,81.895546,201.219025,NaN,47.218590,93.748497,119.868706,330.791901,32.778252,...,124.111427,22.857887,64.638580,118.131447,NaN,118.004425,278.205536,171.030014,107.040466,52.692532
2020-01-06,73.426826,77.802071,82.324615,199.905060,NaN,47.255360,95.143997,119.349228,331.766083,32.731289,...,124.760391,22.881294,64.647408,117.014145,NaN,117.182968,280.136841,169.565231,106.563591,51.972080
2020-01-07,73.081505,77.358223,81.866959,195.589096,NaN,46.988743,95.343002,118.723923,335.285156,32.515270,...,123.342941,22.969076,64.576920,117.222450,NaN,119.444290,278.445740,168.280029,106.380180,51.476772
2020-01-08,74.257103,77.906494,82.200676,195.972763,NaN,47.540359,94.598503,120.773018,329.410095,32.843994,...,124.154121,23.039301,65.079140,116.843704,NaN,119.767349,284.316498,170.122803,106.985458,51.368698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,70.546989,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,82.060143,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,82.758926,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
## doesnt work yet 
# Remove invalid stocks per assignment requirement (ones outside of the U.S & ones with average monthly volume of less than 200,000 shares)

def filter_market(df):
    df_transposed = df.T
    for i in range(len(df_transposed)):
        # Add country of market as a column to dataframe
        df_transposed["Market"].iloc[i] = stocks_full_info[i].info['market']
    
    # Drops the stocks in markets that are not in the U.S
    df_transposed.drop(df_transposed[df_transposed.Market != "us_market"].index, inplace=True)
    df_transposed = df_transposed.drop('Market', axis=1)
    return_df = df.T

    return return_df


def filter_volume(df):

    start = "2022-01-01"
    end = "2022-10-31"

    df_transposed = df.T

    for i in range(len(df_transposed)):
        ticker = yf.Ticker(df_transposed.iloc[i])
        ticker_hist = ticker.history(start=start, end=end)

        volume_hist = ticker_hist.loc[(ticker_hist.index >= pd.to_datetime(start)) & (ticker_hist.index <= pd.to_datetime(end))]
        df_transposed["Volume"].iloc[i] = (volume_hist.info['averageDailyVolume10Day'].mean())*3

    # Drops the stocks with volumes less than average monthly volume of 200,000
    df_transposed.drop(df_transposed[df_transposed.Volume < 200000].index, inplace=True)
    df_transposed = df_transposed.drop('Volume', axis=1)
    return_df = df.T

    return return_df

all_prices = all_prices.loc[filter_market(all_prices)]
all_prices = all_prices.loc[filter_volume(all_prices)]

# Drops stocks with more than specified number of NaN values
max_number_of_nans = 20
all_prices = all_prices.loc[:, (all_prices.isnull().sum(axis=0) <= max_number_of_nans)]


KeyError: 0

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.